In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=a7b5c0b0418458bff0cbd28581c7866e2e97a670954458278c9b230e0b5bc887
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
#import pandas as pd

In [ ]:
#df=pd.read_csv('/content/drive/MyDrive/Datasets/salestxns.csv')

In [ ]:
#df.head()

In [ ]:
#dff=pd.read_csv('/content/drive/MyDrive/Datasets/customers.csv')

In [ ]:
#dff.head()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Initialize Spark session
spark = SparkSession.builder.appName("CustomerSalesAggregation").getOrCreate()

# Step 1: Load Customer Data
df = spark.read \
    .format("csv") \
    .option("header", True) \
    .load("/content/drive/MyDrive/Datasets/customers.csv")

# Step 2: Broadcast Customer Data
broadcast_customer_data = spark.sparkContext.broadcast(
    df.select("Customer_Id", "Name").toPandas()
)

# Step 3: Load Sales Transaction Data (Assuming you have a sales DataFrame named sales_df)
sales_df = spark.read \
    .format("csv") \
    .option("header", True) \
    .load("/content/drive/MyDrive/Datasets/salestxns.csv")

# # Step 4: Join Sales Transaction Data with Broadcasted Customer Data
joined_df = sales_df.join(broadcast_customer_data.value, col("Customer_Id") == col("Customer_Id"))

# # Step 5: Perform Aggregations
aggregated_df = joined_df.groupBy("Customer_Id", "Name", "Product_Id", "Product_Name", "Price") \
    .agg(
        sum("Qty").alias("Total_Quantity"),
        sum("Price").alias("Total_Amount_Paid")
    )

# Step 6: Display the Result
aggregated_df.show(truncate=False)

# Step 7: Stop Spark Session
spark.stop()

AttributeError: 'DataFrame' object has no attribute '_jdf'

In [ ]:
#customer_df

In [ ]:
df = spark.read \
    .format("csv") \
    .option("header", True) \
    .load("/content/drive/MyDrive/Datasets/customers.csv")

In [ ]:
df

DataFrame[Customer_Id: string, Name: string, City: string, State: string, Zipcode: string]

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSVReader") \
    .getOrCreate()

df = spark.read \
    .format("csv") \
    .option("header", True) \
    .option("delimiter", ",") \
    .load("/content/drive/MyDrive/Datasets/customers.csv")

df.show(5)

spark.stop()


+-----------+--------------+---------+-----+-------+
|Customer_Id|          Name|     City|State|Zipcode|
+-----------+--------------+---------+-----+-------+
|      11039|   Mary Torres|   Caguas|   PR|    725|
|       5623|    Jose Haley| Columbus|   OH|  43207|
|       5829|    Mary Smith|  Houston|   TX|  77015|
|       6336|Richard Maddox|   Caguas|   PR|    725|
|       1708|Margaret Booth|Arlington|   TX|  76010|
+-----------+--------------+---------+-----+-------+
only showing top 5 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Initialize Spark session
spark = SparkSession.builder.appName("CustomerSalesAggregation").getOrCreate()

# Step 1: Load Customer Data
customer_df = spark.read \
    .format("csv") \
    .option("header", True) \
    .load("/content/drive/MyDrive/Datasets/customers.csv")

# Step 2: Create a lookup table (optimized for joins)
customer_lookup = customer_df.select("Customer_Id", col("Name").alias("Customer_Name")).cache()

# Step 3: Load Sales Transaction Data
sales_df = spark.read \
    .format("csv") \
    .option("header", True) \
    .load("/content/drive/MyDrive/Datasets/salestxns.csv")

# Step 4: Join using the lookup table (avoiding broadcast)
joined_df = sales_df.join(customer_lookup, on="Customer_Id")

# Step 5: Perform Aggregations
# aggregated_df = joined_df.groupBy("Customer_Id", "Customer_Name", "Product_Id", "Product_Name", "Price") \
#     .agg(
#         sum("Qty").alias("Total_Quantity"),
#         sum("Price").alias("Total_Amount_Paid")
#     )
aggregated_df = joined_df.groupBy("Customer_Id", "Customer_Name", "Product_Id", "Product_Name", "Price") \
    .agg(
        sum("Qty").alias("Total_Quantity"),
        sum(col("Qty") * col("Price")).alias("Total_Amount_Paid")
    )

# Step 6: Display the Result
aggregated_df.show(truncate=False)

# Step 7: Stop Spark Session
spark.stop()


+-----------+----------------+----------+---------------------------------------------+------+--------------+------------------+
|Customer_Id|Customer_Name   |Product_Id|Product_Name                                 |Price |Total_Quantity|Total_Amount_Paid |
+-----------+----------------+----------+---------------------------------------------+------+--------------+------------------+
|7491       |Sharon Dominguez|502       |Nike Men's Dri-FIT Victory Golf Polo         |50    |11.0          |550.0             |
|1518       |Jessica Smith   |191       |Nike Men's Free 5.0+ Running Shoe            |99.99 |14.0          |1399.86           |
|11845      |Mary Weaver     |1014      |O'Brien Men's Neoprene Life Vest             |49.98 |4.0           |199.92            |
|712        |Mary Ayala      |1004      |Field & Stream Sportsman 16 Gun Fire Safe    |399.98|2.0           |799.96            |
|8757       |Tyler Gutierrez |627       |Under Armour Girls' Toddler Spine Surge Runni|39.99 |7.0